In [1]:
from bs4 import BeautifulSoup
import requests
import time
from urllib.request import urlopen
import ssl
import re
import pandas as pd
import gspread
from gspread_dataframe import set_with_dataframe
from os.path import splitext, basename

In [2]:
url='https://www.espncricinfo.com/records/tournament/team-match-results/icc-cricket-world-cup-2023-24-15338'

r= requests.get(url)

In [3]:
r

<Response [200]>

In [4]:
soup= BeautifulSoup(r.text,"html.parser")

table = soup.find("table", class_=lambda x: x and "ds-w-full" in x)


In [5]:
print(table)

<table class="ds-w-full ds-table ds-table-xs ds-table-auto ds-w-full ds-overflow-scroll ds-scrollbar-hide"><thead class="ds-bg-fill-content-alternate ds-text-left"><tr class=""><td class="ds-min-w-max"><div class="ds-popper-wrapper"><span class="ds-cursor-pointer">Team 1</span></div></td><td class="ds-min-w-max ds-text-right"><div class="ds-popper-wrapper"><span class="ds-cursor-pointer">Team 2</span></div></td><td class="ds-min-w-max ds-text-right"><div class="ds-popper-wrapper"><span class="ds-cursor-pointer">Winner</span></div></td><td class="ds-min-w-max ds-text-right"><div class="ds-popper-wrapper"><span class="ds-cursor-pointer">Margin</span></div></td><td class="ds-min-w-max ds-text-right"><div class="ds-popper-wrapper"><span class="ds-cursor-pointer">Ground</span></div></td><td class="ds-min-w-max ds-text-right ds-whitespace-nowrap"><div class="ds-popper-wrapper"><span class="ds-cursor-pointer">Match Date</span></div></td><td class="ds-min-w-max ds-text-right"><div class="ds-po

In [6]:
header = table.find("thead")

In [7]:
header.text

'Team 1Team 2WinnerMarginGroundMatch DateScorecard'

In [8]:
rows=header.find_all("td")

head=[]
for i in range(len(rows)):
    print(rows[i])

<td class="ds-min-w-max"><div class="ds-popper-wrapper"><span class="ds-cursor-pointer">Team 1</span></div></td>
<td class="ds-min-w-max ds-text-right"><div class="ds-popper-wrapper"><span class="ds-cursor-pointer">Team 2</span></div></td>
<td class="ds-min-w-max ds-text-right"><div class="ds-popper-wrapper"><span class="ds-cursor-pointer">Winner</span></div></td>
<td class="ds-min-w-max ds-text-right"><div class="ds-popper-wrapper"><span class="ds-cursor-pointer">Margin</span></div></td>
<td class="ds-min-w-max ds-text-right"><div class="ds-popper-wrapper"><span class="ds-cursor-pointer">Ground</span></div></td>
<td class="ds-min-w-max ds-text-right ds-whitespace-nowrap"><div class="ds-popper-wrapper"><span class="ds-cursor-pointer">Match Date</span></div></td>
<td class="ds-min-w-max ds-text-right"><div class="ds-popper-wrapper"><span class="ds-cursor-pointer">Scorecard</span></div></td>


In [9]:
rows=header.find_all("td")

head=[]
for i in range(len(rows)):
    
    soup1=rows[i]
    x=soup1.find("span",class_="ds-cursor-pointer")
    print(x)
    head.append(x.text)

<span class="ds-cursor-pointer">Team 1</span>
<span class="ds-cursor-pointer">Team 2</span>
<span class="ds-cursor-pointer">Winner</span>
<span class="ds-cursor-pointer">Margin</span>
<span class="ds-cursor-pointer">Ground</span>
<span class="ds-cursor-pointer">Match Date</span>
<span class="ds-cursor-pointer">Scorecard</span>


In [10]:
head

['Team 1', 'Team 2', 'Winner', 'Margin', 'Ground', 'Match Date', 'Scorecard']

In [11]:
table

<table class="ds-w-full ds-table ds-table-xs ds-table-auto ds-w-full ds-overflow-scroll ds-scrollbar-hide"><thead class="ds-bg-fill-content-alternate ds-text-left"><tr class=""><td class="ds-min-w-max"><div class="ds-popper-wrapper"><span class="ds-cursor-pointer">Team 1</span></div></td><td class="ds-min-w-max ds-text-right"><div class="ds-popper-wrapper"><span class="ds-cursor-pointer">Team 2</span></div></td><td class="ds-min-w-max ds-text-right"><div class="ds-popper-wrapper"><span class="ds-cursor-pointer">Winner</span></div></td><td class="ds-min-w-max ds-text-right"><div class="ds-popper-wrapper"><span class="ds-cursor-pointer">Margin</span></div></td><td class="ds-min-w-max ds-text-right"><div class="ds-popper-wrapper"><span class="ds-cursor-pointer">Ground</span></div></td><td class="ds-min-w-max ds-text-right ds-whitespace-nowrap"><div class="ds-popper-wrapper"><span class="ds-cursor-pointer">Match Date</span></div></td><td class="ds-min-w-max ds-text-right"><div class="ds-po

In [12]:
body=table.find("tbody")
rows=body.find_all("tr")

data = []
for row in rows:
    temp = []
    # print(row)
    tdata = row.find_all("td", class_="ds-min-w-max")
    for td in tdata:
        temp.append(td.text)

    data.append(temp)


In [13]:
data

[['India',
  'Australia',
  'Australia',
  '6 wickets',
  'Ahmedabad',
  'Nov 19, 2023',
  'ODI # 4705'],
 ['Australia',
  'South Africa',
  'Australia',
  '3 wickets',
  'Eden Gardens',
  'Nov 16, 2023',
  'ODI # 4704'],
 ['India',
  'New Zealand',
  'India',
  '70 runs',
  'Wankhede',
  'Nov 15, 2023',
  'ODI # 4703'],
 ['India',
  'Netherlands',
  'India',
  '160 runs',
  'Bengaluru',
  'Nov 12, 2023',
  'ODI # 4702'],
 ['England',
  'Pakistan',
  'England',
  '93 runs',
  'Eden Gardens',
  'Nov 11, 2023',
  'ODI # 4701'],
 ['Australia',
  'Bangladesh',
  'Australia',
  '8 wickets',
  'Pune',
  'Nov 11, 2023',
  'ODI # 4700'],
 ['Afghanistan',
  'South Africa',
  'South Africa',
  '5 wickets',
  'Ahmedabad',
  'Nov 10, 2023',
  'ODI # 4699'],
 ['New Zealand',
  'Sri Lanka',
  'New Zealand',
  '5 wickets',
  'Bengaluru',
  'Nov 9, 2023',
  'ODI # 4698'],
 ['England',
  'Netherlands',
  'England',
  '160 runs',
  'Pune',
  'Nov 8, 2023',
  'ODI # 4697'],
 ['Afghanistan',
  'Australia'

In [14]:
df=pd.DataFrame(data,columns=head)

In [15]:
df

,Team 1,Team 2,Winner,Margin,Ground,Match Date,Scorecard
0,India,Australia,Australia,6 wickets,Ahmedabad,"Nov 19, 2023",ODI # 4705
1,Australia,South Africa,Australia,3 wickets,Eden Gardens,"Nov 16, 2023",ODI # 4704
2,India,New Zealand,India,70 runs,Wankhede,"Nov 15, 2023",ODI # 4703
3,India,Netherlands,India,160 runs,Bengaluru,"Nov 12, 2023",ODI # 4702
4,England,Pakistan,England,93 runs,Eden Gardens,"Nov 11, 2023",ODI # 4701
5,Australia,Bangladesh,Australia,8 wickets,Pune,"Nov 11, 2023",ODI # 4700
6,Afghanistan,South Africa,South Africa,5 wickets,Ahmedabad,"Nov 10, 2023",ODI # 4699
7,New Zealand,Sri Lanka,New Zealand,5 wickets,Bengaluru,"Nov 9, 2023",ODI # 4698
8,England,Netherlands,England,160 runs,Pune,"Nov 8, 2023",ODI # 4697
9,Afghanistan,Australia,Australia,3 wickets,Wankhede,"Nov 7, 2023",ODI # 4696


In [36]:
body=table.find("tbody")
rows=body.find_all("tr")

data_1 = []
for row in rows:
    temp = []
    # print(row)
    tdata = row.find_all("td", class_="ds-min-w-max")
    for j in range(len(tdata)):
        if j <6:
            temp.append(tdata[j].text)
        else:
            temp.append(tdata[j].text)
            temp.append(tdata[j].find("a").get("href"))
    

    data_1.append(temp)


In [37]:
data_1

[['India',
  'Australia',
  'Australia',
  '6 wickets',
  'Ahmedabad',
  'Nov 19, 2023',
  'ODI # 4705',
  '/series/icc-cricket-world-cup-2023-24-1367856/india-vs-australia-final-1384439/full-scorecard'],
 ['Australia',
  'South Africa',
  'Australia',
  '3 wickets',
  'Eden Gardens',
  'Nov 16, 2023',
  'ODI # 4704',
  '/series/icc-cricket-world-cup-2023-24-1367856/australia-vs-south-africa-2nd-semi-final-1384438/full-scorecard'],
 ['India',
  'New Zealand',
  'India',
  '70 runs',
  'Wankhede',
  'Nov 15, 2023',
  'ODI # 4703',
  '/series/icc-cricket-world-cup-2023-24-1367856/india-vs-new-zealand-1st-semi-final-1384437/full-scorecard'],
 ['India',
  'Netherlands',
  'India',
  '160 runs',
  'Bengaluru',
  'Nov 12, 2023',
  'ODI # 4702',
  '/series/icc-cricket-world-cup-2023-24-1367856/india-vs-netherlands-45th-match-1384436/full-scorecard'],
 ['England',
  'Pakistan',
  'England',
  '93 runs',
  'Eden Gardens',
  'Nov 11, 2023',
  'ODI # 4701',
  '/series/icc-cricket-world-cup-2023-2

In [39]:
head

['Team 1', 'Team 2', 'Winner', 'Margin', 'Ground', 'Match Date', 'Scorecard']

In [40]:
df_1=pd.DataFrame(data_1,columns=['Team 1', 'Team 2', 'Winner', 'Margin', 'Ground', 'Match Date', 'id','Scorecard'])

In [41]:
df_1['Scorecard']=df_1['Scorecard'].apply(lambda x:"https://www.espncricinfo.com/"+x)

In [42]:
df_1['Scorecard'][1]

'https://www.espncricinfo.com//series/icc-cricket-world-cup-2023-24-1367856/australia-vs-south-africa-2nd-semi-final-1384438/full-scorecard'

In [45]:
df_1['id']=df_1['id'].apply(lambda x:x[-4:])

In [66]:
df_1

,Team 1,Team 2,Winner,Margin,Ground,Match Date,id,Scorecard
0,India,Australia,Australia,6 wickets,Ahmedabad,"Nov 19, 2023",4705,https://www.espncricinfo.com//series/icc-crick...
1,Australia,South Africa,Australia,3 wickets,Eden Gardens,"Nov 16, 2023",4704,https://www.espncricinfo.com//series/icc-crick...
2,India,New Zealand,India,70 runs,Wankhede,"Nov 15, 2023",4703,https://www.espncricinfo.com//series/icc-crick...
3,India,Netherlands,India,160 runs,Bengaluru,"Nov 12, 2023",4702,https://www.espncricinfo.com//series/icc-crick...
4,England,Pakistan,England,93 runs,Eden Gardens,"Nov 11, 2023",4701,https://www.espncricinfo.com//series/icc-crick...
5,Australia,Bangladesh,Australia,8 wickets,Pune,"Nov 11, 2023",4700,https://www.espncricinfo.com//series/icc-crick...
6,Afghanistan,South Africa,South Africa,5 wickets,Ahmedabad,"Nov 10, 2023",4699,https://www.espncricinfo.com//series/icc-crick...
7,New Zealand,Sri Lanka,New Zealand,5 wickets,Bengaluru,"Nov 9, 2023",4698,https://www.espncricinfo.com//series/icc-crick...
8,England,Netherlands,England,160 runs,Pune,"Nov 8, 2023",4697,https://www.espncricinfo.com//series/icc-crick...
9,Afghanistan,Australia,Australia,3 wickets,Wankhede,"Nov 7, 2023",4696,https://www.espncricinfo.com//series/icc-crick...


## Score Card

In [46]:
df_1[:3]['Scorecard']

0    https://www.espncricinfo.com//series/icc-crick...
1    https://www.espncricinfo.com//series/icc-crick...
2    https://www.espncricinfo.com//series/icc-crick...
Name: Scorecard, dtype: object

In [80]:
import pandas as pd
from bs4 import BeautifulSoup
import requests

def scrape_details(url):
    r = requests.get(url)
    soup = BeautifulSoup(r.text, "html.parser")

    section = soup.find("section", id="main-container")
    batting_team = []
    bowling_team = []

    if section:
        divs = section.find_all("div", class_="ds-rounded-lg ds-mt-2")

        for div in divs:
            country = div.find("span", "ds-capitalize").text

            # Scrape batting details
            bat_table = div.find("table", "ci-scorecard-table")
            body = bat_table.find("tbody")
            scorecard_rows = body.find_all("tr")

            for row in scorecard_rows:
                temp_score = []
                if "ds-hidden" not in row.get("class", []):
                    columns = row.find_all("td")
                    if len(columns) >= 7 and columns[0].find("a") is not None:
                        player_name = columns[0].find("a").get("title")
                        if player_name:
                            temp_score.extend([country, player_name] + [col.text for col in columns[2:]])

                            # Handling the additional 'out' column
                            out_column = row.find("td", class_="ds-min-w-max !ds-pl-[100px]")
                            if out_column:
                                span = out_column.find("span", class_="ds-items-center")
                                temp_score.append(span.text if span else "null")
                            else:
                                temp_score.append("null")

                            batting_team.append(temp_score)

            # Scrape bowling details
            bowl_table = div.find("table", "ds-w-full ds-table ds-table-md ds-table-auto")
            body = bowl_table.find("tbody")
            scorecard_rows = body.find_all("tr")

            for row in scorecard_rows:
                temp_score = []
                if "ds-hidden" not in row.get("class", []):
                    columns = row.find_all("td")
                    temp_score.append(country)
                    temp_score.append(columns[0].find("span").text)
                    temp_score.extend([col.text for col in columns[1:]])

                    bowling_team.append(temp_score)

    return batting_team, bowling_team

final_batting_summary_list = []
final_bowling_summary_list = []

# Iterate over URLs
for i in range(len(df_1)):
    url1 = df_1.loc[i, 'Scorecard']

    print(url1)
    batting_team1, bowling_team1 = scrape_details(url1)

    batting_df = pd.DataFrame(batting_team1, columns=['country', 'Batsman', 'Runs', 'Balls', 'Minutes', 'fours', 'sixes', 'SR', 'out'])
    batting_df['id'] = df_1.loc[i, 'id']
    final_batting_summary_list.append(batting_df)

    bowling_df = pd.DataFrame(bowling_team1, columns=['country', 'Bowler', 'Overs', 'Maidens', 'Runs', 'Wickets', 'Econ', '0s', '4s', '6s', 'WD', 'NB'])
    bowling_df['id'] = df_1.loc[i, 'id']
    final_bowling_summary_list.append(bowling_df)

# Concatenate all DataFrames in the list
final_batting_summary = pd.concat(final_batting_summary_list, ignore_index=True)
final_bowling_summary = pd.concat(final_bowling_summary_list, ignore_index=True)

# Print the final DataFrames
print(final_batting_summary)
print(final_bowling_summary)


https://www.espncricinfo.com//series/icc-cricket-world-cup-2023-24-1367856/india-vs-australia-final-1384439/full-scorecard
https://www.espncricinfo.com//series/icc-cricket-world-cup-2023-24-1367856/australia-vs-south-africa-2nd-semi-final-1384438/full-scorecard
https://www.espncricinfo.com//series/icc-cricket-world-cup-2023-24-1367856/india-vs-new-zealand-1st-semi-final-1384437/full-scorecard
https://www.espncricinfo.com//series/icc-cricket-world-cup-2023-24-1367856/india-vs-netherlands-45th-match-1384436/full-scorecard
https://www.espncricinfo.com//series/icc-cricket-world-cup-2023-24-1367856/england-vs-pakistan-44th-match-1384435/full-scorecard
https://www.espncricinfo.com//series/icc-cricket-world-cup-2023-24-1367856/australia-vs-bangladesh-43rd-match-1384434/full-scorecard
https://www.espncricinfo.com//series/icc-cricket-world-cup-2023-24-1367856/afghanistan-vs-south-africa-42nd-match-1384433/full-scorecard
https://www.espncricinfo.com//series/icc-cricket-world-cup-2023-24-1367856/

In [83]:
df_batting=pd.read_csv("batting_data.csv")

In [84]:
df_batting

,Player,Country,Span,Mat,Inns,NO,Runs,HS,Ave,SR,100,50,0
0,SA Abbott,Australia,2023-2023,1,-,0,-,-,-,0.00,-,-,-
1,Abdullah Shafique,Pakistan,2023-2023,8,8,0,336,113,42.00,93.33,1,3,1
2,CN Ackermann,Netherlands,2023-2023,9,9,0,216,69,24.00,82.12,-,1,1
3,Agha Salman,Pakistan,2023-2023,3,1,0,51,51,51.00,113.33,-,1,-
4,MM Ali,England,2023-2023,6,6,0,95,42,15.83,74.80,-,-,-
...,...,...,...,...,...,...,...,...,...,...,...,...,...
146,CR Woakes,England,2023-2023,8,8,1,131,51,18.71,84.51,-,1,1
147,MA Wood,England,2023-2023,7,7,4,85,43*,28.33,130.76,-,-,1
148,SA Yadav,India,2023-2023,7,7,1,106,49,17.66,100.95,-,-,-
149,WA Young,New Zealand,2023-2023,6,6,0,206,70,34.33,83.73,-,2,1


In [85]:
df_batting[df_batting['Player'].isin(final_bowling_summary['Bowler'])]

,Player,Country,Span,Mat,Inns,NO,Runs,HS,Ave,SR,100,50,0
3,Agha Salman,Pakistan,2023-2023,3,1,0,51,51,51.00,113.33,-,1,-
8,Azmatullah Omarzai,Afghanistan,2023-2023,9,8,3,353,97*,70.60,97.78,-,3,-
31,Fazalhaq Farooqi,Afghanistan,2023-2023,6,3,2,2,2*,2.00,22.22,-,-,1
34,Haris Rauf,Pakistan,2023-2023,9,5,2,53,35,17.66,117.77,-,-,-
35,Hasan Ali,Pakistan,2023-2023,6,3,0,20,12,6.66,71.42,-,-,1
36,Hasan Mahmud,Bangladesh,2023-2023,2,1,0,15,15,15.00,60.00,-,-,-
44,Iftikhar Ahmed,Pakistan,2023-2023,9,8,2,142,40,23.66,115.44,-,-,-
57,Kuldeep Yadav,India,2023-2023,11,2,1,19,10,19.00,61.29,-,-,-
65,Mahedi Hasan,Bangladesh,2023-2023,3,3,1,33,17,16.50,45.20,-,-,-
66,Mahmudullah,Bangladesh,2023-2023,8,7,1,328,111,54.66,91.62,1,1,-


In [86]:
df_batting[df_batting['Player']=='Mitchell Starc']

,Player,Country,Span,Mat,Inns,NO,Runs,HS,Ave,SR,100,50,0


In [87]:
final_bowling_summary.rename(columns={'country':'innings'},inplace=True)

In [88]:
final_batting_summary.rename(columns={'country':'innings'},inplace=True)

In [89]:
final_batting_summary.to_csv('batting_score_card_all_matches.csv',index=False)

In [90]:
final_bowling_summary.to_csv('bowling_score_card_all_matches.csv',index=False)

In [91]:
final_batting_summary

,innings,Batsman,Runs,Balls,Minutes,fours,sixes,SR,out,id
0,India,Rohit Sharma,47,31,44,4,3,151.61,c Head b Maxwell,4705
1,India,Shubman Gill,4,7,21,0,0,57.14,c Zampa b Starc,4705
2,India,Virat Kohli,54,63,99,4,0,85.71,b Cummins,4705
3,India,Shreyas Iyer,4,3,3,1,0,133.33,c †Inglis b Cummins,4705
4,India,KL Rahul,66,107,133,1,0,61.68,c †Inglis b Starc,4705
...,...,...,...,...,...,...,...,...,...,...
871,England,Adil Rashid,15,13,23,0,1,115.38,null,4658
872,England,Mark Wood,13,14,19,0,0,92.85,null,4658
873,New Zealand,Devon Conway,152,121,147,19,3,125.61,null,4658
874,New Zealand,Will Young,0,1,5,0,0,0.00,c †Buttler b Curran,4658


# Bowling